In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timezone, timedelta

def extract_flight_data():
    """
    Extracts flight data from the OpenSky API for the last 24 hours 
    while handling API restrictions (max 2-hour query windows).
    
    Returns:
        pd.DataFrame: A DataFrame containing flight data.
    """

    # Load credentials from template.env
    load_dotenv(".env")

    USERNAME = os.getenv("OPENSKY_USERNAME")
    PASSWORD = os.getenv("OPENSKY_PASSWORD")

    if not USERNAME or not PASSWORD:
        raise ValueError("Missing OpenSky credentials. Check your template.env file.")

    # Define time range (last 24 hours) using timezone-aware datetime
    end_time = int(datetime.now(timezone.utc).timestamp())   # Current UTC time
    start_time = end_time - 86400  # 24 hours ago

    # Initialize empty list to store data
    all_flights = []

    # Loop through 2-hour intervals (to comply with OpenSky limits)
    interval = 7200  # 2 hours in seconds
    current_start = start_time

    while current_start < end_time:
        current_end = min(current_start + interval, end_time)  # Ensure we don't exceed the last timestamp
        
        # OpenSky API URL for 2-hour window
        url = f"https://opensky-network.org/api/flights/all?begin={current_start}&end={current_end}"

        # Fetch data from OpenSky API
        response = requests.get(url, auth=(USERNAME, PASSWORD))

        # Check for successful response
        if response.status_code == 200:
            flights = response.json()
            all_flights.extend(flights)  # Append new results to list
        else:
            print(f"Error {response.status_code}: {response.text}")
        
        # Move to next 2-hour window
        current_start = current_end

    # Convert collected data to DataFrame
    df_flights = pd.DataFrame(all_flights)

    return df_flights

# Run the function and display first 20 rows
df_flights = extract_flight_data()
df_flights.head(20)
